In [1]:
#######
## if you are unfamiliar with jupyter notebook, take a look at this tutorial
## https://www.dataquest.io/blog/jupyter-notebook-tutorial/
##
## make sure all dependencies are installed

import open3d as o3d # http://www.open3d.org/
import numpy as np
import time
import random
import matplotlib.pyplot as plt
import math
import multiprocessing
from joblib import Parallel, delayed
import pandas
import seaborn as sns
import xlwt
from tempfile import TemporaryFile
print(f"open3d :{o3d.__version__}")

open3d :0.11.0


In [2]:
import platform
print(platform.python_version())

3.8.3


In [3]:
def normalize(vec):
    """normalize a vector"""
    vec = np.asarray(vec)
    return vec/np.linalg.norm(vec)

def getCameraVec(camera_pos=(2, 2, 2), lookat=(0, 0, 0), up=(0, 0, 1)):
    """
    compute camera vector. 
    ref: http://www.songho.ca/opengl/gl_camera.html
    input:
        camera_pos: position of the camera
        lookat: position of the target
        up: camera up vector, usually pointing towards +z
    """
    camera_pos = np.asarray(camera_pos)
    lookat = np.asarray(lookat)
    up = np.asarray(up)
    front = camera_pos - lookat
    zoom = np.linalg.norm(front)
    front = normalize(front)
    left = normalize(np.cross(up, front))
    up = normalize(np.cross(front, left))
    return zoom,front, lookat, up

def setView(ctr,camera_pos=(2, 2, 2), lookat=(0, 0, 0), up=(0, 0, 1)):
    """
    set the view given a view control handel ctr
    """
    zoom,front, lookat, up = getCameraVec(camera_pos=(1, 1, 1), lookat=(0, 0, 0), up=(0, 0, 1))
    ctr.set_constant_z_far(100) # camera z far clip plane
    ctr.set_constant_z_near(0.01)# camera z near clip plane
    ctr.set_front(front)
    ctr.set_lookat(lookat)
    ctr.set_up(up)
    ctr.set_zoom(zoom)
    
def customVisualization(geomtry_list):
    """
    helper function to create a visualization given a list of o3d geometry
    """
    vis = o3d.visualization.Visualizer()
    vis.create_window()
    for g in geomtry_list:
        vis.add_geometry(g)
    ctr = vis.get_view_control()
    setView(ctr)
    vis.run()
    vis.destroy_window() # close the window when finished

In [4]:
def create_grid(bounds=[[-1, -1, -1], [1, 1, 1]], dr=0.1):
    """
    retrun a grid of points shaped (n,3) given bounds and discritization radius
    the bounds are updated and also returned
    input:
        bounds: [(x_low,y_low,z_low),(x_high,y_high,z_high)]
        dr: discretization radius of the grid
    output:
        xyz_grid: a grid of points numpy array of (n,3)
        bounds: updated bounds
        nx,ny,nz: number of points in x,y,z direction
    """
    # round to integer, type is still float
    bounds = bounds/dr
    bounds = np.stack((np.floor(bounds[0]), np.ceil(bounds[1])))*dr
#     print("bounds=\n", bounds)
    # number of points in x,y,z direction:(nx,ny,nz)
    nx, ny, nz = np.ceil((bounds[1]-bounds[0])/dr).astype(int)+1
    x = np.linspace(bounds[0, 0], bounds[0, 0]+(nx-1)*dr, num=nx)
    y = np.linspace(bounds[0, 1], bounds[0, 1]+(ny-1)*dr, num=ny)
    z = np.linspace(bounds[0, 2], bounds[0, 2]+(nz-1)*dr, num=nz)
    # a flattened grid of xyzs of the vertices
    xyz_grid = np.stack(np.meshgrid(x, y, z), axis=-1).reshape((-1, 3))
    return xyz_grid, bounds, (nx, ny, nz)


def create_plane(r=5, dr=0.1):
    """
    return a plane located at (0,0,0),and with plane normal = (0,0,1)
    r: radius of the plane
    dr:discretization radius of the grid
    """
    bounds = np.stack((np.floor([-r, -r, 0]), np.ceil([r, r, 0])))
    nx, ny, nz = np.ceil((bounds[1]-bounds[0])/dr).astype(int)+1

    xyz = np.reshape([[[[i, j, 0], [i+1, j, 0], [i, j+1, 0],
                       [i, j+1, 0], [i+1, j, 0], [i+1, j+1, 0]] for i in range(nx-1)] for j in range(ny-1)], (-1, 3))
    xyz = (xyz - ((nx-1)/2,(ny-1)/2,0))*dr
#     xyz, bounds, (nx, ny, nz) = create_grid(bounds, dr)
#     print(nx, ny, nz)
    triangles = np.arange(xyz.shape[0]).reshape((-1,3))
    plane = o3d.geometry.TriangleMesh(o3d.utility.Vector3dVector(
        xyz), o3d.utility.Vector3iVector(triangles))
    # assign checkerboard color pattern
    c0 = (0.729, 0.78, 0.655) # first color
    c1 = (0.533, 0.62, 0.506) # second color
    colors = np.reshape([[np.tile(c0 if (i+j)%2 else c1,(6,1)) for i in range(nx-1)] for j in range(ny-1)],(-1,3))
    plane.vertex_colors = o3d.utility.Vector3dVector(colors)
    plane.compute_triangle_normals()
    return plane

# create a plane
plane = create_plane()
# create a coordinate frame
coord_frame = o3d.geometry.TriangleMesh.create_coordinate_frame(size=0.1, origin=[0, 0, 0])

# o3d.visualization.draw_geometries([plane, coord_frame],mesh_show_wireframe=True)
#customVisualization([plane, coord_frame])

In [5]:
### My Code Below

In [6]:
T = 0
gravity = np.array([0,0,-9.81])
dt = 0.00001
kg = 100000
#bouncing k
#ks = 10000
damping = 0.9900
m = 1
mus = 1
muk = 0.8
population_size = 30


In [7]:
dt_b = 0.0001

In [8]:
spring_type = {}
# spring type = [k, b, c]
#spring_type[1] = [5000.0, 0.001, math.pi/2]
#spring_type[2] = [20000.0, 0.002, math.pi/2]
#spring_type[3] = [10000.0, 0.001, 0]
#spring_type[4] = [5000.0, 0.002, math.pi]

spring_type[1] = [1000.0, 0, 0]
spring_type[2] = [25000.0, 0, 0]
spring_type[3] = [5000.0, 0.001, 0]
spring_type[4] = [5000.0, 0.001, math.pi]



In [9]:
print(spring_type)

{1: [1000.0, 0, 0], 2: [25000.0, 0, 0], 3: [5000.0, 0.001, 0], 4: [5000.0, 0.001, 3.141592653589793]}


In [87]:
class mass:
    def __init__(self, n):
        self.mass = np.zeros((n,1), dtype=np.float64)
        self.position = np.zeros((n,3), dtype=np.float64)
        self.velocity = np.zeros((n,3), dtype=np.float64)
        self.acceleration = np.zeros((n,3), dtype=np.float64)
        self.internalforce = np.zeros((n,3), dtype=np.float64)
        self.externalforce = np.zeros((n,3), dtype=np.float64)

In [88]:
class spring:
    def __init__(self, n):
        self.coe =  np.zeros((n,1), dtype=np.int32)
        self.length =  np.zeros((n,1), dtype=np.float64)
        self.idx = np.zeros((n,2), dtype=np.int32)
        # add spring type, type number saved in dictionary 
        self.type = [0] * n

In [214]:
def get_points(position_matrix, Vertexs):
    for i in range(len(position_matrix)):
        Vertexs.mass[i] = 0.1
        Vertexs.position[i] = position_matrix[i].copy()
    
def get_edges(Vertexs, Edges
             ):    
    idx = 0
    for i in range(len(Vertexs.mass) - 1):
        first_poisiton = Vertexs.position[i]
        for j in range(i + 1, len(Vertexs.mass)):
            secondposition = Vertexs.position[j]
            distance = np.linalg.norm(first_poisiton - secondposition)
            #Edges.coe[idx] = ks
            Edges.length[idx] = distance
            Edges.idx[idx] = [i,j]
            # randomly assign spring types
            Edges.type[idx] = random.randint(1, 5)
            idx += 1 

In [215]:
def centroid(points):
    x_coords = np.average(points.position[:,0])
    y_coords = np.average(points.position[:,1])
    #z_coords = np.average(points.position[:,2])

    return np.linalg.norm((x_coords, y_coords))

In [216]:
def bouncing(time, springlist, Vertexs, Edges):
    
    for j in range(len(Edges.idx)):
        t = springlist[j]
        b, c, = spring_type[t][1], spring_type[t][2]
        Edges.length[j] = b*(np.sin(1500*time + c) ) * Edges.length[j] + Edges.length[j]

    for i in range(len(Vertexs.mass)):
        for j in range(len(Edges.idx)):
            if Edges.idx[j][0] != i:
                continue
            h = springlist[j]
            ks = spring_type[h][0]
            first_idx, second_idx = Edges.idx[j][0], Edges.idx[j][1]
            first_idx_distance = Vertexs.position[Edges.idx[j][1]] - Vertexs.position[i]
            second_idx_distance = Vertexs.position[i] - Vertexs.position[Edges.idx[j][1]]
            distance_norm = np.linalg.norm((second_idx_distance))
            deltal = distance_norm - Edges.length[j]           
            Force_scalar = ks * deltal
            
            Vertexs.internalforce[first_idx] += Force_scalar * normalize(first_idx_distance)
            Vertexs.internalforce[second_idx] += Force_scalar * normalize(second_idx_distance)
            
        Vertexs.externalforce[i] += Vertexs.mass[i] * gravity
        totalforce = Vertexs.internalforce[i] + Vertexs.externalforce[i]
        if Vertexs.position[i][2] <= 0:
            FH = np.sqrt( np.square(totalforce[0]) + np.square(totalforce[1]) )
            FY = totalforce[2]
            if FH <= abs(FY * mus):
                Vertexs.velocity[i][0], Vertexs.velocity[i][1] = 0.0, 0.0
            else:
                totalforce[0] -= totalforce[0] / FH * abs(FY * muk)
                totalforce[1] -= totalforce[1]/ FH * abs(FY * muk)
            totalforce += kg * np.array([0,0,-Vertexs.position[i][2]])
            #totalforce *= 0.9
            
        Vertexs.acceleration[i] = (totalforce) / Vertexs.mass[i]
        Vertexs.velocity[i] += Vertexs.acceleration[i] * dt_b 
        Vertexs.velocity[i] = Vertexs.velocity[i] * damping
        Vertexs.position[i] += Vertexs.velocity[i]*dt_b 
        Vertexs.internalforce[i] = np.array([0,0,0])
        Vertexs.externalforce[i] = np.array([0,0,0])
            
    return       
        
      

In [217]:
# 下面两个是 选择的function：

In [218]:

def wheelselection(ordered_map):
    pp=[]
    p=[]
    possible=0.3
    for i in range(population_size):
        pp.append((possible*(pow((1-possible),(i-1)))))
        bigger=2*pp[i]
        p.append(bigger)
        
    randnum=random.random()
    if randnum > p[1]:
        parent_id=ordered_map[0][1]
        move_id=ordered_map[0][2]
    elif randnum > p[2] and randnum < p[1]:
        parent_id=ordered_map[1][1]
        move_id=ordered_map[1][2]
    elif randnum > p[3] and randnum <p[2]:
        parent_id=ordered_map[2][1]
        move_id=ordered_map[2][2]
    elif randnum > p[4] and randnum <p[3]:
        parent_id=ordered_map[3][1]
        move_id=ordered_map[3][2]
    elif randnum > p[5] and randnum <p[4]:
        parent_id=ordered_map[4][1]
        move_id=ordered_map[4][2]
    elif randnum > p[6] and randnum <p[5]:
        parent_id=ordered_map[5][1]
        move_id=ordered_map[5][2]
    elif randnum > p[7] and randnum <p[6]:
        parent_id=ordered_map[6][1]
        move_id=ordered_map[6][2]
    elif randnum > p[8] and randnum <p[7]:
        parent_id=ordered_map[7][1]
        move_id=ordered_map[7][2]
    elif randnum > p[9] and randnum <p[8]:
        parent_id=ordered_map[8][1]
        move_id=ordered_map[8][2]
    elif randnum > p[10] and randnum <p[9]:
        parent_id=ordered_map[9][1]
        move_id=ordered_map[9][2]
    elif randnum > p[11] and randnum <p[10]:
        parent_id=ordered_map[10][1]
        move_id=ordered_map[10][2]
    elif randnum > p[12] and randnum <p[11]:
        parent_id=ordered_map[11][1]
        move_id=ordered_map[11][2]
    elif randnum > p[13] and randnum <p[12]:
        parent_id=ordered_map[12][1]
        move_id=ordered_map[12][2]
    elif randnum > p[14] and randnum <p[13]:
        parent_id=ordered_map[13][1]
        move_id=ordered_map[13][2]
    elif randnum > p[15] and randnum <p[14]:
        parent_id=ordered_map[14][1]
        move_id=ordered_map[14][2]
    elif randnum > p[16] and randnum <p[15]:
        parent_id=ordered_map[15][1]
        move_id=ordered_map[15][2]
    elif randnum > p[17] and randnum <p[16]:
        parent_id=ordered_map[16][1]
        move_id=ordered_map[16][2]
    elif randnum > p[18] and randnum <p[17]:
        parent_id=ordered_map[17][1]
        move_id=ordered_map[17][2]
    elif randnum > p[19] and randnum <p[18]:
        parent_id=ordered_map[18][1]
        move_id=ordered_map[18][2]
    else:
        parent_id=ordered_map[19][1]
        move_id=ordered_map[19][2]
    
        
    offspring=parent_id
    newposition = move_id
    
    return offspring, newposition

In [219]:
def top50_selection(processed_list, population):
    selectedpop=[]
    lst=[]
    c=sorted(processed_list)
    d=c[0:15]
    d=np.array(d)
    for i in range(population_size):
        if processed_list[i] > d.all():
            selectedpop.append(population[i])
    R=np.random.randint(15)
    selectedpop=np.array(selectedpop)
    lst=selectedpop(R)
    return lst

In [243]:
def mutation(springlist):
    cur = random.randint(1, 4)
    target = random.randint(1,4)
    
    for i in range(len(springlist)):
        if springlist[i] == cur:
            springlist[i] = target
            
    return springlist

In [244]:
def mutation_position(position):
    #n
    idx = random.randint(0, len(position)-1)
    if position[idx][2] == 0.0:
        for i in range(len(position[idx]) - 1):
            p = random.random()
            if p >= 0.5:
                position[idx][i] = random.uniform(0.01, 0.04)
    else:
        for i in range(len(position[idx])):
            p = random.random()
            if p >= 0.5:
                position[idx][i] = random.uniform(0.01, 0.04)   
    return position

In [255]:
def mutation_numbermass(individual,offspring1):
    if len(individual) < 8:
        x=random.uniform(0.01, 0.04)
        y=random.uniform(0.01, 0.04)
        z=random.uniform(0.01, 0.04)
        newpoistion=[x, y, z]
        individual.append(newpoistion)
        numberx=len(individual)
        changelengt=(numberx*(numberx-1))//2
        numl=len(offspring1)
        addlength=changelengt-numl
        
        for i in range(addlength):
            idx = random.randint(1, 4)
            offspring1.append(idx)
        return

In [256]:
def crossover(lst1, lst2):
    #idx = random.randint(17, 27)
    idx = random.randint(0, 6)
    offspring1 = lst2[:idx] + lst1[idx:]
    offspring2 = lst1[:idx] + lst2[idx:]
    
    return offspring1, offspring2

In [257]:
def diversity(population):
    visited = set()
    total = len(population)
    for i in range(len(population)):
        if tuple(population[i]) in visited:
            total -= 1 
        visited.add(tuple(population[i]))
    return total

In [258]:
def diff_population(processed_list):
    dotp=[]
    dotp=processed_list
    dotp=list(set(dotp))
    return dotp

In [259]:
# 我设置了0.1秒， 每个population跑0.1， centriod fuction是计算中心的， 每次和开始的中心距离作对比
def calculatefitness(springlist, bouncing_position):
    T = 0
    Vertexs = mass( len(bouncing_position) )
    Edges = spring( (len(bouncing_position) * (len(bouncing_position) - 1)) // 2)
    get_points(bouncing_position, Vertexs)
    get_edges(Vertexs,Edges )
    Initial = centroid(Vertexs)
    while  T < 0.02:
        bouncing(T, springlist, Vertexs, Edges)
        T += dt_b
    distance = abs(centroid(Vertexs) - Initial)
    return distance

In [260]:
Initial_position =  [[0.025,0.025,0.0],
                      [0.025,0,0.0],
                      [0,0,0.0],
                      [0,0.025,0.0],
                      [0.0125, 0.0125, 0.025]]

In [261]:
from copy import deepcopy
population_size = 20
population = []
moving_position = []
for _ in range(population_size):
    springcollection = []
    for _ in range(10):
        idx = random.randint(1, 4)
        springcollection.append(idx)
    population.append(springcollection)
    moving_position.append(deepcopy(Initial_position))
#print(population)

In [262]:
    
runtimes=5
fitness=0
fitness = []
D = []
get_dot=[]
dt_b = 0.0001
for i in range(10):

    
    
    #moving_position = [[0.025,0.025,0.0],
    #                  [0.025,0,0.0],
    #                  [0,0,0.0],
    #                  [0,0.025,0.0],
    #                   [0.0125, 0.0125, 0.025]]
    
    num_cores = multiprocessing.cpu_count()
    processed_list = Parallel(n_jobs=num_cores)(delayed(calculatefitness)(population[i], moving_position[i] ) for i in range(len(population)))
    mapping = list(zip(processed_list, population, moving_position))
    ttordered_map = sorted(mapping, key = lambda x: x[0], reverse = True)
    fitness.append(ttordered_map[0][0])
    linkage=ttordered_map[0][1]
    point_position=ttordered_map[0][2]
    dotp=diff_population(processed_list)

        
    population = []
    moving_position=[]
    for sequence in ttordered_map[:6]:
        population.append(sequence[1].copy())
        moving_position.append(deepcopy(sequence[2]))
        
  
    ##----------------------------------------------
    ##----------------------------------------------changed code below
   
    ordered_map=ttordered_map[:]
           
    for j in range(7):
        
        #selection50%
        #offspring1 = top50_selection(processed_list, population)
        #offspring2 = top50_selection(processed_list, population)
        
        # wheelselection 
        offspring1, MovingP1 = wheelselection(deepcopy(ordered_map))
        offspring2, MovingP2 = wheelselection(deepcopy(ordered_map))
        #print(id(MovingP1[2]))
        #print(id(MovingP2[2]))
        
        offspring1, offspring2  = crossover(offspring1, offspring2)
        
        offspring1 = mutation(offspring1)
        offspring2 = mutation(offspring2)
                
        P1 = mutation_position(deepcopy(MovingP1))
        P2 = mutation_position(deepcopy(MovingP2))
        
        
        changerate=0.5
        changenumber=random.random()
        if changerate >= changenumber:
            mutation_numbermass(P1,offspring1)
        else:
            mutation_numbermass(P2,offspring2)
        
        #change=random.
        
        

        #print(P1)  
        #print('-----------------------------------')
        #print(P2) 
        
        population.append(offspring1)
        population.append(offspring2)
        moving_position.append(deepcopy(P1))
        moving_position.append(deepcopy(P2))
    ##----------------------------------------------
    ##----------------------------------------------changed code above
    
    #for x in range(2):
        
    
    
    #for z in range(2):
       # n=np.randint(3,8)
        #totalposition=[]
        #for i in range(n):
            #newpoistion=[]
            #x=random.uniform(0.01, 0.04)
           # y=random.uniform(0.01, 0.04)
           # z=random.uniform(0.01, 0.04)
            #newpoistion=[[x],[y],[z]]
            #totalposition.append(newposition)
       # moving_position.append(totalposition)
    
    d = diversity(population.copy())
    D.append(d)
    print(i)
#print(moving_position)

0
1
2
3
4
5
6
7
8
9


In [263]:
for i in range(len(population)):
    print(population[i])

[4, 3, 4, 3, 4, 3, 3, 3, 4, 3, 4, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 3, 3, 4, 3, 3, 3]
[4, 3, 3, 3, 4, 3, 3, 3, 4, 3, 4, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 3, 3, 4, 3, 3, 3]
[4, 3, 3, 3, 4, 3, 3, 3, 4, 3, 4, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 3, 3, 4, 3, 3, 3]
[4, 3, 3, 3, 4, 3, 3, 3, 4, 3, 4, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 3, 3, 4, 3, 3, 3]
[4, 3, 3, 3, 4, 3, 3, 3, 4, 3, 4, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 3, 3, 4, 3, 3, 3]
[4, 3, 3, 3, 4, 3, 3, 3, 4, 3, 4, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 3, 3, 4, 3, 3, 3]
[4, 3, 3, 3, 4, 3, 3, 3, 4, 3, 4, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 3, 3, 4, 3, 3, 3]
[2, 3, 3, 3, 2, 3, 3, 3, 2, 3, 2, 3, 3, 3, 3, 3, 3, 3, 3, 2, 2, 2, 3, 3, 2, 3, 3, 3]
[1, 3, 3, 3, 1, 3, 3, 3, 1, 3, 1, 3, 3, 3, 3, 3, 3, 3, 3, 1, 1, 1, 3, 3, 1, 3, 3, 3]
[2, 3, 3, 3, 2, 3, 3, 3, 2, 3, 2, 3, 3, 3, 3, 3, 3, 3, 3, 2, 2, 2, 3, 3, 2, 3, 3, 3]
[4, 1, 1, 1, 4, 1, 1, 1, 4, 1, 4, 1, 1, 1, 1, 1, 1, 1, 1, 4, 4, 4, 1, 1, 4, 1, 1, 1]
[4, 3, 3, 3, 4, 3, 3, 3, 4, 3, 4, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4

In [264]:
for i in range(len(moving_position)):
    print(len(moving_position[i]))

8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8


In [265]:
print(fitness)

[8.617829019355941e-05, 0.00012680145356852926, 0.0002352199340229337, 0.0002352199340229337, 0.0002536699319191024, 0.0002536699319191024, 0.0002817196902809953, 0.0002817196902809953, 0.0002817196902809953, 0.0002997636175455995]


In [288]:
print(linkage)

[4, 3, 4, 3, 4, 3, 3, 3, 4, 3, 4, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 3, 3, 4, 3, 3, 3]


In [289]:
print(population)

[[4, 3, 4, 3, 4, 3, 3, 3, 4, 3, 4, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 3, 3, 4, 3, 3, 3], [4, 3, 3, 3, 4, 3, 3, 3, 4, 3, 4, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 3, 3, 4, 3, 3, 3], [4, 3, 3, 3, 4, 3, 3, 3, 4, 3, 4, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 3, 3, 4, 3, 3, 3], [4, 3, 3, 3, 4, 3, 3, 3, 4, 3, 4, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 3, 3, 4, 3, 3, 3], [4, 3, 3, 3, 4, 3, 3, 3, 4, 3, 4, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 3, 3, 4, 3, 3, 3], [4, 3, 3, 3, 4, 3, 3, 3, 4, 3, 4, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 3, 3, 4, 3, 3, 3], [4, 3, 3, 3, 4, 3, 3, 3, 4, 3, 4, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 3, 3, 4, 3, 3, 3], [2, 3, 3, 3, 2, 3, 3, 3, 2, 3, 2, 3, 3, 3, 3, 3, 3, 3, 3, 2, 2, 2, 3, 3, 2, 3, 3, 3], [1, 3, 3, 3, 1, 3, 3, 3, 1, 3, 1, 3, 3, 3, 3, 3, 3, 3, 3, 1, 1, 1, 3, 3, 1, 3, 3, 3], [2, 3, 3, 3, 2, 3, 3, 3, 2, 3, 2, 3, 3, 3, 3, 3, 3, 3, 3, 2, 2, 2, 3, 3, 2, 3, 3, 3], [4, 1, 1, 1, 4, 1, 1, 1, 4, 1, 4, 1, 1, 1, 1, 1, 1, 1, 1, 4, 4, 4, 1, 1, 4, 1, 1, 1], [4, 3, 3, 3, 4, 3, 3, 3, 4, 3, 4, 3, 3, 3, 3, 3, 3, 3

In [290]:
print(D)

[20, 18, 17, 19, 19, 13, 8, 7, 6, 7]


In [291]:
print(ttordered_map[0][2])

[[0.021070760577750258, 0.010349212257700404, 0.0], [0.025, 0, 0.0], [0, 0, 0.0], [0.01940975370201881, 0.025, 0.0], [0.03311129374004002, 0.0125, 0.025], [0.03719994120907174, 0.020519253742647497, 0.0297120413651435], [0.03215636831318789, 0.02771681771928814, 0.0139184799430051], [0.034337848734023416, 0.03868678916171439, 0.017728704997043376]]


In [292]:
print(get_dot)

[]


In [293]:
linkage 

[4,
 3,
 4,
 3,
 4,
 3,
 3,
 3,
 4,
 3,
 4,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 4,
 4,
 4,
 3,
 3,
 4,
 3,
 3,
 3]

In [294]:
point_position

[[0.021070760577750258, 0.010349212257700404, 0.0],
 [0.025, 0, 0.0],
 [0, 0, 0.0],
 [0.01940975370201881, 0.025, 0.0],
 [0.03311129374004002, 0.0125, 0.025],
 [0.03719994120907174, 0.020519253742647497, 0.0297120413651435],
 [0.03215636831318789, 0.02771681771928814, 0.0139184799430051],
 [0.034337848734023416, 0.03868678916171439, 0.017728704997043376]]

In [295]:
Vertexs = mass( len(point_position) )
Edges = spring( len(point_position) * (len(point_position) - 1) // 2)
get_points(point_position, Vertexs)
get_edges(Vertexs,Edges )

In [296]:
initial_distance = centroid(Vertexs)
print(initial_distance)

0.03038377542501824


In [298]:
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(Vertexs.position)
lsd = o3d.geometry.LineSet()
lsd.points = o3d.utility.Vector3dVector(Vertexs.position) # (n by 3) arry of points xyzs
#lsd_colors = (pcd_colors[edges[:,0]]+pcd_colors[edges[:,1]])/2
#lsd.colors = o3d.utility.Vector3dVector(lsd_colors)
lsd.lines = o3d.utility.Vector2iVector(Edges.idx)
alpha = 0.03
mesh = o3d.geometry.TriangleMesh.create_from_point_cloud_alpha_shape(pcd, alpha)
#mesh.compute_vertex_normals()
#customVisualization([pcd, lsd, coord_frame,plane])

In [299]:
dt_b = 0.0003

In [300]:
def NonBlockingVisualization(kinetic, potential, springE):
    """
    example for the non-blocking visualization
    """    
    ended=False
    def signalEnd(vis):
        nonlocal ended
        ended =True
        
    vis = o3d.visualization.VisualizerWithKeyCallback()
    
    # ref:https://www.glfw.org/docs/3.3/group__keys.html
    # press key Q or ESC to close the window
    vis.register_key_callback(81, signalEnd)# key Q 
    vis.register_key_callback(256, signalEnd)# key escape
    vis.create_window()
    
    # change background to black
    opt = vis.get_render_option()
    opt.background_color = np.asarray([0, 0, 0])

    # add geometry
    for g in [pcd, lsd, coord_frame,plane]:
        vis.add_geometry(g)
        
    # view control
    ctr = vis.get_view_control()
    setView(ctr)
    T = 0
     # previous time
    t_prev = time.time()
    count = 0
    while (not ended):
        bouncing(T, linkage, Vertexs, Edges)
        count += 1
        t = time.time()
        if t - t_prev>1./30.:# update renderer
            t_prev = t
            lsd.points = o3d.utility.Vector3dVector(Vertexs.position)
            vis.update_geometry(lsd)
            pcd.points = o3d.utility.Vector3dVector(Vertexs.position)
            vis.update_geometry(pcd)
            vis.poll_events()
            vis.update_renderer()
        T += dt

    vis.destroy_window() # close the window when finished
    
    return count

In [301]:
### New code above
NonBlockingVisualization([],[],[])


3955

In [ ]:
[3, 1, 3, 1, 4, 4, 3, 3, 1, 4, 3, 1, 4, 3, 4, 3, 3, 3, 3, 3, 4, 3, 3, 3, 3, 4, 4, 4]

In [ ]:
Totalenergy = np.add(np.add(K,P), S)

In [ ]:
print(len(K))

In [ ]:
plt.figure(figsize=(8,5))
#plt.plot(K, label='kinetic')
#plt.plot(P, label='potential')
#plt.plot(S, label='spring')
plt.plot(Totalenergy, label='Total')
plt.xlabel('Time(10e-4)')
plt.ylabel('Energy')
plt.title('Time and Energy plot')
plt.legend()
print(K[:3])

In [ ]:
def breathing(time):  
    SE = 0
    PE = 0
    KE = 0
    
    for i in range(len(Vertexs.mass)):
        for j in range(len(Edges.idx)):
            if Edges.idx[j][0] != i:
                continue
            first_idx, second_idx = Edges.idx[j][0], Edges.idx[j][1]
            first_idx_distance = Vertexs.position[Edges.idx[j][1]] - Vertexs.position[i]
            second_idx_distance = Vertexs.position[i] - Vertexs.position[Edges.idx[j][1]]
            Edges.length[j] += 0.005*(np.sin(1000*time))
            distance_norm = np.linalg.norm((first_idx_distance))
            deltal = distance_norm - Edges.length[j]
            SE += calculate_spring_energy(first_idx, deltal, ks)
            SE += calculate_spring_energy(second_idx, deltal, ks)
            Force_scalar = ks * deltal
            Vertexs.internalforce[first_idx] += Force_scalar * normalize(first_idx_distance)
            Vertexs.internalforce[second_idx] += Force_scalar * normalize(second_idx_distance)
            
        
        Vertexs.acceleration[i] = (Vertexs.internalforce[i]) / Vertexs.mass[i]
        Vertexs.velocity[i] += Vertexs.acceleration[i] * dt
        Vertexs.position[i][:2]+= Vertexs.velocity[i][:2]*dt
        Vertexs.internalforce[i] = np.array([0,0,0])    
        P, K = calculate_energy(i)
        PE += P
        KE += K
            
    return SE, PE, KE  

def calculate_energy(i):
    Potential = Vertexs.mass[i] * 9.81 * Vertexs.position[i][2]
    Kinectic = 0.5 * Vertexs.mass[i] * np.linalg.norm((Vertexs.velocity[i]))**2
    
    return Potential, Kinectic

def calculate_spring_energy(idx, dl, ks):
    Spring = 0.5 * ks * dl**2
    return Spring